MovieLens Analysis Pipeline with DuckDB + DagsHub
This project demonstrates how to build a generalizable, scalable, and reproducible data pipeline for exploratory analysis of the MovieLens 10M dataset, leveraging modern open‑source tools and cloud‑based hosting.

Pipeline Steps
- Data Extraction (.dat)
The original MovieLens files (ratings.dat, movies.dat, tags.dat) are read directly from DagsHub URLs. DuckDB’s read_csv_auto parses the "::" delimited files. Options like ignore_errors=true and quote='' handle malformed lines and problematic quotes.
- Transformation to Parquet
After ingestion, the data is converted into Parquet files: ratings.parquet, movies.parquet, tags.parquet.
Parquet is a columnar format, which means that instead of storing data row by row, it stores it column by column. This makes analytical queries much faster, reduces disk usage through compression, and allows selective reading of only the columns needed for a query.
- Upload to DagsHub
The generated Parquet files are uploaded back to the remote repository on DagsHub. This ensures that the processed, optimized data is available to anyone collaborating on the project, without requiring them to repeat the extraction and transformation steps.
- Direct Queries on Remote Parquet
DuckDB can query Parquet files directly from DagsHub via HTTP, without downloading them locally. This means that analysis can be performed on demand, from any machine, simply by pointing to the remote file URLs.
- Exploratory Data Analysis
SQL queries performed include: record counts per table, rating distribution, top 10 most‑rated movies, top 10 highest‑rated movies with at least 100 ratings, genre popularity, most active users, and most frequent tags.

Data Structure
- ratings: userId, movieId, rating, timestamp
- movies: movieId, title, genres
- tags: userId, movieId, tag, timestamp

Running Queries Online
You can run queries directly against the Parquet files hosted on DagsHub, without downloading. For example, in Python with DuckDB you can create views from the remote Parquet files and run SQL queries to retrieve insights.

Advantages of Each Component
Cloud Storage (DagsHub Data Hosting)
- Accessibility: Data stored in the cloud can be accessed from anywhere with an internet connection, removing the need to manually copy or transfer files between machines.
- Collaboration: Multiple team members can work on the same dataset without duplicating storage or worrying about version mismatches.
- Versioning: DagsHub integrates with Git and DVC, so datasets can be versioned just like code, ensuring reproducibility of experiments.
- Reliability: Cloud storage reduces the risk of data loss due to local hardware failures.
Git
- Version Control for Code and Notebooks: Tracks every change to scripts and Jupyter notebooks, ensuring that exploratory steps, visualizations, and analysis logic are preserved.
- Processed Data Control: By linking processed data artifacts (such as Parquet files) to commits, Git ensures that the exact version of the data used for analysis can be reproduced.
- Collaboration: Multiple contributors can work on the same project without overwriting each other’s work.
DagsHub
- Unified Platform: Combines Git, DVC, MLflow, and issue tracking in one place.
- Data + Code Integration: Hosts both the code repository and the dataset, ensuring they stay in sync.
- Remote Access: Provides raw file URLs that can be read directly by analytical tools like DuckDB.
- Experiment Tracking: Supports logging and comparing experiments, making it easier to manage machine learning workflows.
DuckDB
- Lightweight Database: Runs in‑process, meaning no server setup is required.
- Analytical Power: Optimized for OLAP (analytical) queries, making it ideal for exploring large datasets.
- Direct File Access: Can query Parquet, CSV, and JSON files directly, including those hosted remotely.
- SQL Interface: Provides a familiar SQL syntax, lowering the barrier for analysts and data scientists.
- Exploratory Analysis Advantage: Because it integrates seamlessly with notebooks, DuckDB allows interactive queries, quick aggregations, and joins across large datasets without leaving the analysis environment. This makes it perfect for iterative exploration and hypothesis testing.


4. Consultas diretas nos Parquets remotos
- O DuckDB acessa os arquivos Parquet direto do link do DagsHub, sem precisar baixá-los manualmente.
- São criadas views (ratings, movies, tags) a partir dos Parquets remotos.
5. Análise Exploratória
Consultas SQL realizadas:
- Contagem de registros em cada tabela.
- Distribuição de notas.
- Top 10 filmes mais avaliados.
- Top 10 filmes com melhor média (mínimo 100 avaliações).
- Popularidade por gênero.
- Usuários mais ativos.
- Tags mais frequentes.

📂 Estrutura dos Dados
- ratings: userId, movieId, rating, timestamp
- movies: movieId, title, genres
- tags: userId, movieId, tag, timestamp

🖥️ Como Rodar as Consultas Online
Você pode rodar as consultas diretamente nos arquivos Parquet hospedados no DagsHub, sem precisar baixar nada.
Exemplo em Python


import duckdb

base_url = "https://dagshub.com/Matheuskcode/Big-Data-Found/raw/main/data"

files = {
    "ratings": f"{base_url}/ratings.parquet",
    "movies": f"{base_url}/movies.parquet",
    "tags": f"{base_url}/tags.parquet"
}

con = duckdb.connect()

con.execute(f"CREATE OR REPLACE VIEW ratings AS SELECT * FROM parquet_scan('{files['ratings']}')")
con.execute(f"CREATE OR REPLACE VIEW movies  AS SELECT * FROM parquet_scan('{files['movies']}')")
con.execute(f"CREATE OR REPLACE VIEW tags    AS SELECT * FROM parquet_scan('{files['tags']}')")

df = con.execute("""
    SELECT m.title, COUNT(r.rating) as n_ratings, AVG(r.rating) as avg_rating
    FROM ratings r
    JOIN movies m ON r.movieId = m.movieId
    GROUP BY m.title
    ORDER BY n_ratings DESC
    LIMIT 5
""").df()

print(df)

✅ Benefícios do Pipeline
• 	Generalizável: qualquer pessoa pode rodar em qualquer computador, basta ter Python + DuckDB.
• 	Sem download manual: os dados são lidos direto do DagsHub via HTTP.
• 	Eficiente: uso de Parquet acelera consultas e economiza espaço.
• 	Exploratório: consultas SQL permitem análises rápidas e flexíveis.



---

### 💻 Código Python (pipeline completo)

```python
import duckdb
import pandas as pd

# --- 1. Definir URLs dos arquivos no DagsHub (.dat) ---
base_url = "https://dagshub.com/Matheuskcode/Big-Data-Found/raw/main/data"

files = {
    "ratings": f"{base_url}/ratings.dat",
    "movies": f"{base_url}/movies.dat",
    "tags": f"{base_url}/tags.dat"
}

con = duckdb.connect()

# --- 2. Ler arquivos .dat direto da URL ---
ratings = con.execute(f"""
    SELECT * FROM read_csv_auto('{files['ratings']}',
        delim='::',
        columns={{'userId':'BIGINT','movieId':'BIGINT','rating':'DOUBLE','timestamp':'BIGINT'}},
        ignore_errors=true,
        quote=''
    )
""").df()

movies = con.execute(f"""
    SELECT * FROM read_csv_auto('{files['movies']}',
        delim='::',
        columns={{'movieId':'BIGINT','title':'VARCHAR','genres':'VARCHAR'}},
        ignore_errors=true,
        quote=''
    )
""").df()

tags = con.execute(f"""
    SELECT * FROM read_csv_auto('{files['tags']}',
        delim='::',
        columns={{'userId':'BIGINT','movieId':'BIGINT','tag':'VARCHAR','timestamp':'BIGINT'}},
        ignore_errors=true,
        quote=''
    )
""").df()

# --- 3. Salvar em Parquet ---
ratings.to_parquet("ratings.parquet", index=False)
movies.to_parquet("movies.parquet", index=False)
tags.to_parquet("tags.parquet", index=False)

# --- 4. Upload para DagsHub (executar em notebook com dagshub-cli instalado) ---
# !dagshub upload Matheuskcode/Big-Data-Found "ratings.parquet" data/ratings.parquet --update
# !dagshub upload Matheuskcode/Big-Data-Found "movies.parquet" data/movies.parquet --update
# !dagshub upload Matheuskcode/Big-Data-Found "tags.parquet" data/tags.parquet --update

# --- 5. Consultas diretas nos Parquets remotos ---
files_parquet = {
    "ratings": f"{base_url}/ratings.parquet",
    "movies": f"{base_url}/movies.parquet",
    "tags": f"{base_url}/tags.parquet"
}

con.execute(f"CREATE OR REPLACE VIEW ratings AS SELECT * FROM parquet_scan('{files_parquet['ratings']}')")
con.execute(f"CREATE OR REPLACE VIEW movies  AS SELECT * FROM parquet_scan('{files_parquet['movies']}')")
con.execute(f"CREATE OR REPLACE VIEW tags    AS SELECT * FROM parquet_scan('{files_parquet['tags']}')")

# --- 6. Análises Exploratórias ---
print("Tamanhos:")
print(con.execute("SELECT COUNT(*) AS n_ratings FROM ratings").df())
print(con.execute("SELECT COUNT(*) AS n_movies FROM movies").df())
print(con.execute("SELECT COUNT(*) AS n_tags FROM tags").df())

print("\nDistribuição de notas:")
print(con.execute("""
    SELECT rating, COUNT(*) as freq
    FROM ratings
    GROUP BY rating
    ORDER BY rating
""").df())

print("\nTop 10 filmes mais avaliados:")
print(con.execute("""
    SELECT m.title, COUNT(r.rating) as n_ratings, AVG(r.rating) as avg_rating
    FROM ratings r
    JOIN movies m ON r.movieId = m.movieId
    GROUP BY m.title
    ORDER BY n_ratings DESC
    LIMIT 10
""").df())

print("\nTop 10 filmes com melhor média (>=100 avaliações):")
print(con.execute("""
    SELECT m.title, COUNT(r.rating) as n_ratings, AVG(r.rating) as avg_rating
    FROM ratings r
    JOIN movies m ON r.movieId = m.movieId
    GROUP BY m.title
    HAVING COUNT(r.rating) >= 100
    ORDER BY avg_rating DESC
    LIMIT 10
""").df())

print("\nPopularidade por gênero:")
print(con.execute("""
    SELECT genre, COUNT(*) as n_movies
    FROM (
        SELECT movieId, UNNEST(STRING_SPLIT(genres, '|')) as genre
        FROM movies
    )
    GROUP BY genre
    ORDER BY n_movies DESC
""").df())

print("\nTop 10 usuários mais ativos:")
print(con.execute("""
    SELECT userId, COUNT(*) as n_ratings, AVG(rating) as avg_rating
    FROM ratings
    GROUP BY userId
    ORDER BY n_ratings DESC
    LIMIT 10
""").df())

print("\nTop 10 tags mais usadas:")
print(con.execute("""
    SELECT LOWER(tag) as tag, COUNT(*) as freq
    FROM tags
    GROUP